In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import copy
from pathlib import Path
import torch

In [ ]:
from src.training.RawDatasetDNG import RawDatasetDNG
from src.training.load_config import load_config


In [ ]:
from src.training.censored_fit import censored_linear_fit_twosided
from pytorch_msssim import ms_ssim
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
run_config = load_config()
dataset_path = Path(run_config['cropped_raw_subdir'])
align_csv = dataset_path / run_config['secondary_align_csv']
new_align_csv = str(align_csv).replace('.csv', '_exposure_corr.csv')
device=run_config['device']

batch_size = run_config['batch_size']
lr = run_config['lr_base'] * batch_size
clipping =  run_config['clipping']

num_epochs = run_config['num_epochs_pretraining']
val_split = run_config['val_split']
crop_size = run_config['crop_size']
experiment = run_config['mlflow_experiment']
model_params = run_config['model_params']
rggb = model_params['rggb']

colorspace = run_config['colorspace']


In [ ]:
crop_size = 1900

In [ ]:
dataset = RawDatasetDNG(dataset_path, align_csv, colorspace, crop_size=crop_size, validation=True, apply_exposure_corr=False)


In [ ]:
def compute_scale_factors(aligned, noisy):
    scale_factors = []
    sigmas = []

    for channel in range(3):
        _, b, sigma = censored_linear_fit_twosided(aligned[channel], noisy[channel], include_offset=False)
        scale_factors.append(b)
        sigmas.append(sigma)
    return scale_factors, sigmas

In [ ]:
row_list = []
for i in tqdm(range(len(dataset))):
    try:
        output = dataset[i]
        scale_factors, sigmas = compute_scale_factors(output['aligned'], output['noisy'])
        row = dataset.df.iloc[i].to_dict()
        row['r_scale_factor'] = scale_factors[0]
        row['g_scale_factor'] = scale_factors[1]
        row['b_scale_factor'] = scale_factors[2]

        row['r_sigma'] = sigmas[0]
        row['g_sigma'] = sigmas[1]
        row['b_sigma'] = sigmas[2]

        aligned, noisy = output['aligned'], output['noisy']
        uncorrected_l1 = nn.functional.l1_loss(aligned, noisy).item()
        row['uncorrected_l1'] = uncorrected_l1
        aligned[0] *= scale_factors[0]
        aligned[1] *= scale_factors[1]
        aligned[2] *= scale_factors[2]

        corrected_l1 = nn.functional.l1_loss(aligned, noisy).item()
        row['corrected_l1'] = corrected_l1

        row_list.append(row)
    except:
        print(i)

In [ ]:
align_csv = str(align_csv).replace('.csv', '_exposure_corr.csv')

In [ ]:
df = pd.DataFrame(row_list)
df.to_csv(new_align_csv)

In [ ]:
df = pd.read_csv(new_align_csv)

In [ ]:
# Comparing the corrected l1 with uncorrected one to verify the difference has reduced
plt.scatter(df.uncorrected_l1, df.corrected_l1)
plt.plot((0, 0.06), (0, 0.06), color='red')

In [ ]:
df.sort_values('r_scale_factor')

In [ ]:
dataset = RawDatasetDNG(dataset_path, new_align_csv, colorspace, crop_size=crop_size, validation=True, apply_exposure_corr=True)

In [ ]:
import numpy as np
mask = dataset.df.r_scale_factor.values > 0 
matching_indices_in_subset = np.nonzero(mask)[0]
matching_indices_in_subset

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Min r_scale_factor

for idx in tqdm(matching_indices_in_subset):
    row = dataset[idx]

    aligned = row['aligned'].permute(1, 2, 0)**(1/2.2)
    noisy = row['noisy'].permute(1, 2, 0)**(1/2.2)

    plt.subplots(1, 3, figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(aligned)

    plt.subplot(1, 3, 2)
    plt.imshow(noisy)
    plt.title(idx)
    plt.subplot(1, 3, 3)
    plt.imshow(aligned-noisy+0.5)
    plt.savefig(f'/Volumes/EasyStore/cropped_raw_comp/{idx}.jpeg')
    plt.clf()


In [ ]:
idx = 79
row = dataset[79]

In [ ]:
plt.subplots(1, 3, figsize=(15, 5))
aligned = row['aligned'].permute(1, 2, 0)**(1/2.2)
noisy = row['noisy'].permute(1, 2, 0)**(1/2.2)
plt.subplot(1, 3, 1)
plt.imshow(aligned)

plt.subplot(1, 3, 2)
plt.imshow(noisy)
plt.title(idx)
plt.subplot(1, 3, 3)
plt.imshow(aligned-noisy+0.5)


In [ ]:
def compute_scale_factors(aligned, noisy):
    scale_factors = []
    offsets = []

    sigmas = []

    for channel in range(3):
        a, b, sigma = censored_linear_fit_twosided(aligned[channel], noisy[channel], include_offset=True)
        scale_factors.append(b)
        offsets.append(a)
        sigmas.append(sigma)
    return scale_factors, offsets, sigmas

In [ ]:

scale_factors, offsets, sigmas = compute_scale_factors(row['aligned'], row['noisy'])

row['r_scale_factor'] = scale_factors[0]
row['g_scale_factor'] = scale_factors[1]
row['b_scale_factor'] = scale_factors[2]

row['r_sigma'] = sigmas[0]
row['g_sigma'] = sigmas[1]
row['b_sigma'] = sigmas[2]

aligned, noisy = row['aligned'], row['noisy']
uncorrected_l1 = nn.functional.l1_loss(aligned, noisy).item()
row['uncorrected_l1'] = uncorrected_l1
aligned[0] *= scale_factors[0]
aligned[0] += offsets[0]
aligned[1] *= scale_factors[1]
aligned[1] += offsets[1]

aligned[2] *= scale_factors[2]
aligned[2] += offsets[2]


In [ ]:
plt.subplots(1, 3, figsize=(15, 5))
aligned = aligned.permute(1, 2, 0)**(1/2.2)
noisy = row['noisy'].permute(1, 2, 0)**(1/2.2)
plt.subplot(1, 3, 1)
plt.imshow(aligned)

plt.subplot(1, 3, 2)
plt.imshow(noisy)
plt.title(idx)
plt.subplot(1, 3, 3)
plt.imshow(aligned-noisy+0.5)